In [12]:
import pandas as pd
import pandas_ta as ta
import config
from oandapyV20 import API
import oandapyV20.endpoints.instruments as instruments
import oandapyV20.endpoints.orders as orders

from datetime import datetime, timezone
import time

In [13]:
# Setup your OANDA connection
client = API(access_token=config.OANDA_API_KEY)

In [14]:
# define variables
timeframe = "M5"
instrument = "GBP_JPY"

In [15]:
def get_candles(tf):
    params = {
        "granularity": tf,
        "price": "A"  # Ask prices
    }

    r = instruments.InstrumentsCandles(instrument=instrument, params=params)
    candles = client.request(r)['candles']

    # Convert to pandas DataFrame
    data = []

    for c in candles:
        if c["complete"]:
            data.append({
                "time": c["time"],
                "open": float(c["ask"]["o"]),
                "high": float(c["ask"]["h"]),
                "low": float(c["ask"]["l"]),
                "close": float(c["ask"]["c"])
            })

    df = pd.DataFrame(data)
    df["time"] = pd.to_datetime(df["time"])
    
    return df

In [16]:
# --- Mean Reversion Indicators (Bollinger Bands + ATR + RSI) ---
def calculate_indicators(df, bb_len=20, bb_std=2.0, atr_len=14):
    import pandas as pd
    try:
        import pandas_ta as ta
    except:
        ta = None

    # ATR
    if ta:
        try:
            df["ATR_14"] = ta.atr(df["high"], df["low"], df["close"], length=atr_len)
        except:
            pass

    if "ATR_14" not in df.columns or df["ATR_14"].isna().all():
        tr1 = (df["high"] - df["low"]).abs()
        tr2 = (df["high"] - df["close"].shift(1)).abs()
        tr3 = (df["low"] - df["close"].shift(1)).abs()
        tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
        df["ATR_14"] = tr.rolling(atr_len, min_periods=atr_len).mean()

    # Bollinger Bands
    basis = df["close"].rolling(bb_len, min_periods=bb_len).mean()
    stdev = df["close"].rolling(bb_len, min_periods=bb_len).std(ddof=0)

    df["BB_BASIS"] = basis
    df["BB_UPPER"] = basis + bb_std * stdev
    df["BB_LOWER"] = basis - bb_std * stdev

    # RSI (oversold filter)
    if ta:
        try:
            df["RSI_14"] = ta.rsi(df["close"], length=14)
        except:
            pass

    if "RSI_14" not in df.columns or df["RSI_14"].isna().all():
        delta = df["close"].diff()
        up = delta.clip(lower=0).rolling(14, min_periods=14).mean()
        down = (-delta.clip(upper=0)).rolling(14, min_periods=14).mean()
        rs = up / down.replace(0, pd.NA)
        df["RSI_14"] = 100 - (100 / (1 + rs))

    return df


In [17]:
# Function to place an order
def place_order(stop_loss, take_profit):
    data = {
        "order": {
            "instrument": instrument,
            "units": 100,
            "type": "MARKET",
            "stopLossOnFill": {"price": f"{stop_loss:.3f}"},
            "takeProfitOnFill": {"price": f"{take_profit:.3f}"},
        }
    }

    r = orders.OrderCreate(config.OANDA_ACCOUNT_ID, data=data)
    client.request(r)
    print(f"Placed order for {instrument} with stop loss at {round(stop_loss, 3)} and take profit at {round(take_profit, 3)}.")

In [18]:
# --- Mean Reversion Strategy (Bollinger Re-entry) ---
def mean_reversion_strategy(df, bb_len=20, bb_std=2.0, atr_mult=1.0, rr=1.5, use_rsi_filter=True):

    if len(df) < bb_len + 3:
        print("Not enough data for mean reversion strategy")
        return

    last = df.iloc[-1]
    prev = df.iloc[-2]
    prev2 = df.iloc[-3]

    # Setup: candle closed BELOW lower band
    oversold_break = prev2["close"] < prev2["BB_LOWER"]

    # Trigger: price closes BACK inside
    reentry = (prev["close"] <= prev["BB_LOWER"]) and (last["close"] > last["BB_LOWER"])

    cond = oversold_break and reentry

    # Optional RSI filter (RSI should be oversold)
    if use_rsi_filter and not pd.isna(prev["RSI_14"]):
        cond = cond and (prev["RSI_14"] < 35)

    if cond:
        entry = float(last["close"])
        atr = float(last["ATR_14"]) if not pd.isna(last["ATR_14"]) else float(df["ATR_14"].dropna().iloc[-1])

        stop_loss = entry - atr_mult * atr
        take_profit = entry + rr * (entry - stop_loss)

        print(f"BUY (Mean Reversion)  Entry={entry:.5f} SL={stop_loss:.5f} TP={take_profit:.5f}")

        # your existing function remains unchanged
        place_order(stop_loss, take_profit)
    else:
        print("Strategy conditions not met")


In [19]:
def run_bot():
    print("Starting trading bot")
    last_checked = None

    while True:
        current_time = datetime.now(timezone.utc)
        
        # Check for a new 15 minute candle
        #if current_time.minute % 15 == 0 and current_time.second < 10:
        if current_time.minute % 5 == 0 and current_time.second < 10:
            # Check if the 15-minute interval has changed since the last check
            if last_checked != current_time.minute:
                print(f"Current time: {current_time}")
                print("Checking for trade signals")
                price = get_candles(timeframe)
                price = calculate_indicators(price)
                mean_reversion_strategy(price)
                last_checked = current_time.minute  # Update to prevent re-triggering

        time.sleep(1)

In [20]:
run_bot()

Starting trading bot
Current time: 2025-11-05 09:40:00.816147+00:00
Checking for trade signals
Strategy conditions not met
Current time: 2025-11-05 09:45:00.021612+00:00
Checking for trade signals
Strategy conditions not met
Current time: 2025-11-05 09:50:00.049703+00:00
Checking for trade signals
Strategy conditions not met
Current time: 2025-11-05 09:55:00.841043+00:00
Checking for trade signals
Strategy conditions not met
Current time: 2025-11-05 10:00:00.411299+00:00
Checking for trade signals
Strategy conditions not met
Current time: 2025-11-05 10:10:00.762553+00:00
Checking for trade signals
Strategy conditions not met


KeyboardInterrupt: 